In [ ]:
import os

from rl.agents.dqn import TrainDQN
from rl.data import get_train_test_val, load_csv, get_behaviour_data
from rl.utils import rounded_dict, split_csv
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras
from keras import regularizers 
import time
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # CPU is faster than GPU on structured data

# Hyper-parameters.


ModuleNotFoundError: No module named 'reg'

In [ ]:
df1 = pd.read_csv('./data/train_scaled.csv')
df2 = pd.read_csv('./data/test_scaled.csv')

In [ ]:
df1.shape

(136844, 84)

In [ ]:
df2.shape

(32583, 84)

In [ ]:
df1.columns

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [ ]:
df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df1['Timestamp'] = df1['Timestamp'].apply(lambda x: time.mktime(time.strptime(x,'%d/%m/%Y %H:%M:%S')))
df2['Timestamp'] = df2['Timestamp'].apply(lambda x: time.mktime(time.strptime(x,'%d/%m/%Y %H:%M:%S')))

In [ ]:
df1.drop(['Flow ID'], axis=1, inplace=True)
df2.drop(['Flow ID'], axis=1, inplace=True)

In [ ]:
x_train, y_train = df1.iloc[:, :-1], df1.iloc[:, -1]
x_test, y_test = df2.iloc[:, :-1], df2.iloc[:, -1]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)

x_train['Label'] = y_train
x_test['Label'] = y_test

x_train = x_train.sample(frac=1)
x_test = x_test.sample(frac=1)

x_train.to_csv('./data/train_scaled.csv', index=False)
x_test.to_csv('./data/test_scaled.csv', index=False)

In [ ]:
df2['Label'].value_counts()

3    7754
2    7449
4    6522
1    6317
0    4541
Name: Label, dtype: int64

In [ ]:
df1['Label'].value_counts()

4    32608
3    31013
2    29792
1    25268
0    18163
Name: Label, dtype: int64

In [ ]:
episodes = 150_000  # Total number of episodes
warmup_steps = 170_000  # Amount of warmup steps to collect data with random policy
memory_length = warmup_steps  # Max length of the Replay Memory
batch_size = 32
collect_steps_per_episode = 2000
collect_every = 500

target_update_period = 800  # Period to overwrite the target Q-network with the default Q-network
target_update_tau = 1  # Soften the target model update
n_step_update = 1

reg = regularizers.l2(0.01)

layers = [Dense(256, activation="relu", kernel_initializer = "he_normal", kernel_regularizer = reg), Dropout(0.2),
          Dense(256, activation="relu", kernel_initializer = "he_normal", kernel_regularizer = reg), Dropout(0.2),
          Dense(256, activation="relu", kernel_initializer = "he_normal", kernel_regularizer = reg), Dropout(0.2),
          Dense(5, activation=None)]  # No activation, pure Q-values

learning_rate = 0.00025  # Learning rate
gamma = 0.1  # Discount factor
min_epsilon = 0.5  # Minimal and final chance of choosing random action
decay_episodes = episodes // 10  # Number of episodes to decay from 1.0 to `min_epsilon``

# {'backdoor': 7842, 'banker': 4875, 'cryptominer': 990, 'deceptor': 4614, 'downloader': 9274, 'normal': 8961, 'pua': 7149, 'ransomware': 2236, 'spyware': 11585}
# {'backdoor': 0.13632096791016235, 'banker': 0.0847442895386434, 'cryptominer': 0.017209609567847582, 'deceptor': 0.08020721065257448, 'downloader': 0.1612140597295136, 'normal': 0.1557730417550325, 'pua': 0.12427424121266906, 'ransomware': 0.03886938080172444, 'spyware': 0.20138719883183256}

# {'backdoor': 0, 'banker': 1, 'cryptominer': 2, 'deceptor': 3, 'downloader': 4, 'normal': 5, 'pua': 6, 'ransomware': 7, 'spyware': 8}


# Dropping categorical columns and columns that have the same value for all rows.
# split_csv(fp="./data1/RansomwareMod.csv", fp_dest="./data1", name="ransomware", test_size=0.2, strat_col="Label")
df = pd.read_csv('./data/train_scaled.csv')
X_train, y_train, X_test, y_test = load_csv("./data/train_scaled.csv", "./data/test_scaled.csv", "Label", df.select_dtypes(include = ['object']).columns.to_list(), normalization=False)
X_train, y_train, X_test, y_test, X_val, y_val = get_train_test_val(X_train, y_train, X_test, y_test,
                                                                    val_frac=0.3)

# no_rows = 2
# import numpy as np
# X_train, y_train, X_test, y_test, X_val, y_val = get_behaviour_data(no_rows, X_train, y_train, X_test, y_test, X_val, y_val)

# model_path = "./models/rl.pkl"
model = TrainDQN(episodes, warmup_steps, learning_rate, gamma, min_epsilon, decay_episodes, target_update_period=target_update_period,
                  target_update_tau=target_update_tau, batch_size=batch_size, collect_steps_per_episode=collect_steps_per_episode,
                  memory_length=memory_length, collect_every=collect_every, n_step_update=n_step_update, model_path=None)

model.compile_model(X_train, y_train, layers)
model.q_net.summary()
h = model.train(X_val, y_val, "Accuracy")
import json
with open('history_rl.json', 'w') as f:
    json.dump(list(map(str, h)), f)
# import matplotlib.pyplot as plt
# plt.plot(h)
# plt.xlabel("epochs")
# plt.ylabel("Loss")
# print(len(h))
# print(h[0], "<<<<<<<")

# plt.savefig("history.png")

stats = model.evaluate(X_test, y_test, X_train, y_train)
print(rounded_dict(stats))


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             multiple                  21504     
_________________________________________________________________
dropout_27 (Dropout)         multiple                  0         
_________________________________________________________________
dense_37 (Dense)             multiple                  65792     
_________________________________________________________________
dropout_28 (Dropout)         multiple                  0         
_________________________________________________________________
dense_38 (Dense)             multiple                  65792     
_________________________________________________________________
dropout_29 (Dropout)         multiple                  0         
_________________________________________________________________
dense_39 (Dense)             multiple                 

AttributeError: Can't pickle local object 'Layer.add_loss.<locals>._tag_callable'